# DART Report Crawling

## 0. imports

In [1]:
%load_ext jupyter_black

In [2]:
import sys

sys.path.append("..")

In [3]:
import os
import re
import requests

from src.OpenDartReader import OpenDartReader
from bs4 import BeautifulSoup

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [5]:
dart = OpenDartReader(os.environ["DART_API_KEY"])

## 1. 공시검색 (모든종목)

- `start`와 `end`는 3개월 이내 범위만 가능함

In [6]:
# 2022-01-01 ~ 2022-01-10 모든 회사의 모든 공시목록 (3,139 건)
df = dart.list(start="2022-01-01", end="2022-01-10")

In [ ]:
len(df)

In [8]:
rcept_no_list = df["rcept_no"].tolist()

## 2. 공시문서 크롤링

In [ ]:
rcept_no = rcept_no_list[3]

# xml_text = dart.document(rcept_no)
doc_df = dart.sub_docs(rcept_no)

In [ ]:
doc_df.head()

In [21]:
url_list = doc_df["url"].tolist()

In [ ]:
document = {}
for idx, row in doc_df.iterrows():
    url = row["url"]
    title = row["title"]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    text = soup.get_text(strip=False)
    text = re.sub(r"\n+", "\n", text)
    text = re.sub(r" {2,}", " ", text)

    document["title"] = title
    document["text"] = text

In [ ]:
print(document["text"])

In [6]:
import dill

with open("./docs_cache/opendartreader_corp_codes_20241209.pkl", "rb") as f:
    corp_codes = dill.load(f)

In [ ]:
corp_codes